We can also assign governed tags to columns too. These are useful from an AI governance perspective, to verify if our agentic respects all the governance boundaries through the rules and policies we've set using Attribute Based Access Control.

- Create a [governed tag](https://docs.databricks.com/aws/en/admin/governed-tags/manage-governed-tags).

- Create [ABAC policy](https://docs.databricks.com/aws/en/data-governance/unity-catalog/abac/policies?language=SQL).

An illustrative example has been provided below:

In [0]:
%python
dbutils.widgets.text("catalog", "", "1. Catalog")
catalog = dbutils.widgets.get("catalog")

dbutils.widgets.text("schema", "", "2. Schema")
schema = dbutils.widgets.get("schema")

dbutils.widgets.text("schema", "", "3. Customer Table")
table = dbutils.widgets.get("cust_table")

# Set fully qualified reference to table name in Python
fqtable = f"{catalog}.{schema}.{table}"

In [0]:
%sql
-- Set fully qualified reference in SQL
DECLARE OR REPLACE VARIABLE fqtable STRING;
DECLARE OR REPLACE VARIABLE fqrowfilter STRING;
DECLARE OR REPLACE VARIABLE fqcolumnmask STRING;

SET VAR fqtable = concat(:catalog, '.', :schema, '.', :cust_table);
SET VAR fqrowfilter = concat(:catalog, '.', :schema, '.', 'row_filter_');
SET VAR fqcolumnmask = concat(:catalog, '.', :schema, '.', 'column_mask_');

In [0]:
%sql
-- Create UDF with row filter logic

CREATE OR REPLACE FUNCTION IDENTIFIER(fqrowfilter || 'female_gender')(gender STRING)
  RETURNS BOOLEAN
  DETERMINISTIC
  COMMENT 'ABAC utility: Show rows only for female gender'
  RETURN gender IN ('F');

CREATE OR REPLACE FUNCTION IDENTIFIER(fqrowfilter || 'male_gender')(gender STRING)
  RETURNS BOOLEAN
  DETERMINISTIC
  COMMENT 'ABAC utility: Show rows only for male gender'
  RETURN gender IN ('M');

CREATE OR REPLACE FUNCTION IDENTIFIER(fqcolumnmask || 'email_address')(email STRING)
RETURNS STRING
COMMENT 'ABAC utility: Mask email local part while preserving domain'
RETURN CASE
    WHEN email IS NULL OR email = '' THEN email
    WHEN email LIKE '%@%' THEN CONCAT(LEFT(email, 0), '***@', SPLIT(email, '@')[1])
    ELSE '***'
    END
;

In [0]:
%sql
-- Create ABAC policy (row filter) containing who to apply the policy to and how
CREATE OR REPLACE POLICY `filter_by_female_gender`
ON TABLE ${catalog}.${schema}.${cust_table}
ROW FILTER ${catalog}.${schema}.`row_filter_female_gender`
TO `derek.huang@databricks.com`
FOR TABLES
MATCH COLUMNS hasTagValue('gender', 'F') AS gender  -- Trigger when 'gender' key has 'F' value
USING COLUMNS (gender); -- Bind tag to the row filter UDF argument, USING COLUMNS clause only applies to row filters only (TBC)

In [0]:
%sql
CREATE OR REPLACE POLICY `filter_by_male_gender`
ON TABLE ${catalog}.${schema}.${cust_table}
ROW FILTER ${catalog}.${schema}.`row_filter_male_gender`
TO `derek.huang@databricks.com`
FOR TABLES
MATCH COLUMNS hasTagValue('gender', 'M') AS gender  -- Trigger when 'gender' key has 'M' value
USING COLUMNS (gender); -- Bind tag to the row filter UDF argument, USING COLUMNS clause only applies to row filters only (TBC)

In [0]:
%sql
-- Create ABAC policy (column masking) containing who to apply the policy to and how
CREATE OR REPLACE POLICY `mask_email_address`
ON TABLE ${catalog}.${schema}.${cust_table}
COMMENT 'Masking email address'
COLUMN MASK ${catalog}.${schema}.`column_mask_email_address`
TO `account users`
FOR TABLES
MATCH COLUMNS hasTag('mask_email') AS email_address  -- Trigger when a column is tagged with 'mask_email'
ON COLUMN email_address; -- Bind tag to the column masking UDF argument, ON COLUMN clause only applies to column masks only (TBC)

In [0]:
%sql

In [0]:
%python
# Only set tag if tag does not exist
res = spark.sql(
    f"""
        SELECT column_name, tag_name, tag_value
        FROM information_schema.column_tags
        WHERE catalog_name = '{catalog_name}'
        AND schema_name = '{schema_name}'
        AND table_name = '{cust_table_name}';
    """
)
if res.filter(res.column_name == "gender").count() < 1:
    spark.sql(
        f"""
            SET TAG ON COLUMN {cust_table_name}.gender `gender` = `F`;
        """
    )

# Show results
res = spark.sql(
    f"""
        SELECT column_name, tag_name, tag_value
        FROM information_schema.column_tags
        WHERE catalog_name = '{catalog_name}'
        AND schema_name = '{schema_name}'
        AND table_name = '{cust_table_name}';
    """
)
display(res.filter(res.column_name == "gender"))

# You can make subsequent changes via the UI.

In [0]:
%sql
SHOW POLICIES ON TABLE ${catalog}.${schema}.customer;

Now we have governance policies built into our tables, we can now set up transformations and/or Genie spaces to create a seamless experience for our business users through the Databricks One interface.